In [7]:
from keras.models import load_model
import cv2
import numpy as np
from tensorflow import keras
from keras.utils import Sequence
from google.colab import drive
drive.mount('/content/drive')
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def csv_image_generator_75(inputPath, bs, mode="train", aug=None):
	# Abre o arquivo csv para leitura
	f = open(inputPath, "r")

	# Loop infinito
	while True:
		# Inicializa vetores de imagens e classes
		images = []
		labels = []

		# Loop ate atingir o BatchSize
		while len(images) < bs:
			# Lê a proxima linha do CSV
			line = f.readline()

			# Se a linha tiver vazia isso indica que chegamos ao
			# fim do arquivo
			if line == "":
				# Reseta o ponteiro do arquivo para o inicio
				# e lê novamente a primeira linha
				f.seek(0)
				line = f.readline()

				# Se estamos avaliando entao temos que sair do loop
				# para garantirmos que nao avaliaremos o modelo 2 vezes
				# em amostras do inicio do arquivo
				if mode == "eval":
					break

			# Extrai a classe da imagem e constroi a mesma
			line = line.strip().split(",")
			label = int(line[0])
			image = np.array([int(x) for x in line[1:]], dtype="uint8")
			image = image.reshape((50, 50, 3))
			image = cv2.resize(image,(75, 75), interpolation = cv2.INTER_CUBIC)

			# Atualiza as listas do Batch
			images.append(image)
			labels.append(label)

		# Trata parametro de Data Augmentation
		if aug is not None:
			(images, labels) = next(aug.flow(np.array(images),
				labels, batch_size=bs))

		# yield the batch to the calling function
		yield (np.array(images), labels)

In [0]:
def csv_image_generator(inputPath, bs, mode="train", aug=None):
	# Abre o arquivo csv para leitura
	f = open(inputPath, "r")

	# Loop infinito
	while True:
		# Inicializa vetores de imagens e classes
		images = []
		labels = []

		# Loop ate atingir o BatchSize
		while len(images) < bs:
			# Lê a proxima linha do CSV
			line = f.readline()

			# Se a linha tiver vazia isso indica que chegamos ao
			# fim do arquivo
			if line == "":
				# Reseta o ponteiro do arquivo para o inicio
				# e lê novamente a primeira linha
				f.seek(0)
				line = f.readline()

				# Se estamos avaliando entao temos que sair do loop
				# para garantirmos que nao avaliaremos o modelo 2 vezes
                # em amostras do inicio do arquivo
				if mode == "eval":
					break

			# Extrai a classe da imagem e constroi a mesma
			line = line.strip().split(",")
			label = int(line[0])
			image = np.array([int(x) for x in line[1:]], dtype="uint8")
			image = image.reshape((50, 50, 3))

			# Atualiza as listas do Batch
			images.append(image)
			labels.append(label)

		# Trata parametro de Data Augmentation
		if aug is not None:
			(images, labels) = next(aug.flow(np.array(images),
				labels, batch_size=bs))

		# yield the batch to the calling function
		yield (np.array(images), labels)

In [0]:
VAL_CSV = "/content/drive/My Drive/TCC/Projeto/dataset/IDC_validation.csv"
BS = 128
NUM_VAL_IMAGES = 0

labels = set()
valLabels = []

# loop no CSV de validacao
f = open(VAL_CSV, "r")
for line in f:
    # Extrai a label da classe, atualiza a lista de labels e incrementa
    # o numero total de imagens de validacao
    label = line.strip().split(",")[0]
    labels.add(label)
    valLabels.append(label)
    NUM_VAL_IMAGES += 1
    
f.close()

In [0]:
# Re-inicializa o gerador de teste para avaliacao
valGen = csv_image_generator(VAL_CSV, BS,
	mode="eval", aug=None)

valGen75 = csv_image_generator_75(VAL_CSV, BS,
	mode="eval", aug=None)

# Carrega os modelos
Xception = load_model('/content/drive/My Drive/TCC/Projeto/Modelos Treinados/Xception/Xception_1.h5')
VGG19 = load_model('/content/drive/My Drive/TCC/Projeto/Modelos Treinados/VGG19/vgg19_1.h5')
ResNet50 = load_model('/content/drive/My Drive/TCC/Projeto/Modelos Treinados/ResNet50/ResNet50_1.h5')
InceptionV3 = load_model('/content/drive/My Drive/TCC/Projeto/Modelos Treinados/Inceptionv3/InceptionV3_1.h5')

# Realiza predicoes no conjunto de teste
predIdxs_VGG19 = VGG19.predict_generator(valGen,
	steps=(NUM_VAL_IMAGES // BS) + 1, use_multiprocessing=True)

predIdxs_ResNet50 = ResNet50.predict_generator(valGen,
	steps=(NUM_VAL_IMAGES // BS) + 1, use_multiprocessing=True)

predIdxs_Xception = Xception.predict_generator(valGen75,
	steps=(NUM_VAL_IMAGES // BS) + 1, use_multiprocessing=True)

predIdxs_InceptionV3 = InceptionV3.predict_generator(valGen75,
	steps=(NUM_VAL_IMAGES // BS) + 1, use_multiprocessing=True)

def get_label(proba):
    """ Função que transforma probabilidades em classe predita de acordo com
        limiar de 0.5 definido"""
    if proba > 0.5:
        return 1
    else:
        return 0

predLabels_VGG19 = [get_label(i) for i in list(predIdxs_VGG19)]
predLabels_ResNet50 = [get_label(i) for i in list(predIdxs_ResNet50)]
predLabels_Xception = [get_label(i) for i in list(predIdxs_Xception)]
predLabels_InceptionV3 = [get_label(i) for i in list(predIdxs_InceptionV3)]

In [0]:
valLabels = np.array(valLabels).astype(int)

In [18]:
print("[Métricas] Avaliando a rede no conjunto de Validação...\n")
print("[VGG19]: ")
print(classification_report(valLabels,predLabels_VGG19))
print("\n")
print("[ResNet50]: ")
print(classification_report(valLabels,predLabels_ResNet50))
print("\n")
print("[Xception]: ")
print(classification_report(valLabels,predLabels_Xception))
print("\n")
print("[InceptionV3]: ")
print(classification_report(valLabels,predLabels_InceptionV3))
print("\n")
print('=========================================================================')
#print("Acuracia Balanceada: {}\n".format(balanced_accuracy_score(testLabels,predLabels)))
#print("ROC-AUC: {}".format(roc_auc_score(testLabels,predLabels)))

[Métricas] Avaliando a rede no conjunto de Validação...

[VGG19]: 
              precision    recall  f1-score   support

           0       0.86      0.90      0.88      5000
           1       0.89      0.86      0.88      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



[ResNet50]: 
              precision    recall  f1-score   support

           0       0.80      0.92      0.86      5000
           1       0.91      0.77      0.83      5000

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



[Xception]: 
              precision    recall  f1-score   support

           0       0.88      0.93      0.90      5000
           1       0.93      0.87      0.90      5000

    accuracy                           0.90     10000
   macro avg       0.90      0.